In [1]:
import requests
import pandas as pd
import json

In [2]:
data = requests.get('https://api.regional-table.region-services.aws.a2z.com/index.json')

In [3]:
json_data = data.content

In [4]:
raw_data = json.loads(json_data)

In [5]:
raw_data.keys()

dict_keys(['metadata', 'prices'])

In [6]:
collect_data = []
for i in raw_data['prices']:
  collect_data.append(i['attributes'])

In [7]:
all_data = pd.DataFrame(collect_data)

In [8]:
all_data.columns = ['Region', 'Service', 'URL']

In [9]:
all_data.head()

,Region,Service,URL
0,ap-east-1,Amazon Translate,https://aws.amazon.com/translate/
1,ap-northeast-1,Amazon Translate,https://aws.amazon.com/translate/
2,ap-northeast-2,Amazon Translate,https://aws.amazon.com/translate/
3,ap-south-1,Amazon Translate,https://aws.amazon.com/translate/
4,ap-southeast-2,Amazon Translate,https://aws.amazon.com/translate/


In [10]:
all_data.to_csv('All Services.csv', index=False)

In [11]:
regional_services = all_data[(all_data['Region'].str.startswith('us', na=False )) & ~(all_data['Region'].str.contains('gov'))].groupby('Region')['Service'].apply(set).to_frame()

In [12]:
regional_services

,Service
Region,
us-east-1,"{Amazon Pinpoint, Amazon WorkSpaces Web, AWS C..."
us-east-2,"{Amazon Pinpoint, AWS Config, Amazon GuardDuty..."
us-west-1,"{AWS Config, Amazon GuardDuty, AWS Snowcone, A..."
us-west-2,"{Amazon Pinpoint, Amazon WorkSpaces Web, AWS C..."


In [13]:
common_services = set.intersection(*regional_services['Service'])

In [14]:
common_df = pd.DataFrame(common_services).rename(columns={0: 'Common US Services'})
common_df.to_csv('Common Services.csv', index=False)

In [15]:
regional_services['Special Service'] = regional_services['Service'].apply(lambda x: x.difference(common_services))

In [16]:
unique_services = regional_services.drop('Service', axis = 1).explode('Special Service')

In [17]:
for i in unique_services.index.unique():
  unique_services.loc[i].to_csv(f'{i}.csv', index=False)


In [18]:
# location_request = requests.get('https://b0.p.awsstatic.com/locations/1.0/aws/current/locations.json')

In [19]:
# location_data = json.loads(location_request.content)

In [20]:
# location_name_list = []
# for k,v in location_data.items():
#   location_name_list.append({'Code': v['code'], 'Name': k})

In [21]:
# location_info = pd.DataFrame(location_name_list)

In [22]:
# location_info.head(5)